In [1]:
import pandas as pd
import numpy as np
import datetime
import scipy.sparse as spa
import cvxpy as cp
import matplotlib.pyplot as plt
import geopandas as gpd
import contextily as ctx

In [2]:
full_data = pd.read_csv("yellow_tripdata_2010-01.csv")
full_data['pickup_datetime'] = pd.to_datetime(full_data['pickup_datetime'])
full_data['dropoff_datetime'] = pd.to_datetime(full_data['dropoff_datetime'])

year = 2010
month = 1
day = 21
date = datetime.date(year, month, day)
midnight = datetime.datetime(year, month, day, 0, 0)

day_data = full_data.loc[(full_data['pickup_datetime'].dt.date == date) & (full_data['dropoff_datetime'].dt.date == date)]
day_data = day_data[['pickup_datetime', 'dropoff_datetime', 'trip_distance', 'pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude', 'fare_amount', 'surcharge', 'mta_tax', 'tip_amount', 'tolls_amount', 'total_amount']]
day_data = day_data.loc[(day_data['pickup_longitude'] != 0) & (day_data['pickup_latitude'] != 0)]

n = 100
data = day_data.sample(n).sort_values(by = 'pickup_datetime')
data.index = range(n)
data

C:\Users\holly\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3147: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,pickup_datetime,dropoff_datetime,trip_distance,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,fare_amount,surcharge,mta_tax,tip_amount,tolls_amount,total_amount
0,2010-01-21 00:14:40,2010-01-21 00:26:06,5.30,-73.992562,40.748288,-73.980820,40.748547,13.7,0.5,0.5,2.00,0.0,16.70
1,2010-01-21 00:14:44,2010-01-21 00:19:58,1.50,-73.987108,40.775537,-73.992817,40.758124,6.1,0.5,0.5,1.00,0.0,8.10
2,2010-01-21 00:53:00,2010-01-21 01:03:00,4.29,-73.956535,40.760785,-73.992158,40.724225,12.1,0.5,0.5,0.00,0.0,13.10
3,2010-01-21 00:55:00,2010-01-21 01:04:00,2.84,-73.982673,40.767737,-73.994447,40.734758,8.5,0.5,0.5,0.00,0.0,9.50
4,2010-01-21 01:21:20,2010-01-21 01:45:24,14.60,-74.009013,40.705030,-73.867480,40.846635,32.5,0.5,0.5,5.02,0.0,38.52
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,2010-01-21 22:34:48,2010-01-21 22:42:15,1.40,-73.985084,40.727680,-73.981793,40.742415,6.5,0.5,0.5,0.00,0.0,7.50
96,2010-01-21 22:38:40,2010-01-21 23:01:05,5.10,-73.950003,40.785595,-74.003956,40.742149,15.7,0.5,0.5,0.00,0.0,16.70
97,2010-01-21 22:47:00,2010-01-21 22:50:00,0.80,-73.983097,40.761130,-73.990382,40.751050,4.1,0.5,0.5,1.00,0.0,6.10
98,2010-01-21 23:15:00,2010-01-21 23:20:00,0.65,-73.991498,40.735178,-74.001248,40.738377,4.5,0.5,0.5,0.00,0.0,5.50


## Vector Formulation

In [3]:
import import_ipynb
from class_TP import TP

importing Jupyter notebook from class_TP.ipynb


### With Time Window Constraints

In [4]:
def to_td(minutes):
    td = []
    for m in minutes:
        td.append(datetime.timedelta(minutes = m))
    return td

In [5]:
window_mins = [0, 1]
time_windows = to_td(window_mins)
run_times = []
objectives = []

for time_window in time_windows:
    
    tp = TP(data)
    x, problem = tp.problem(time_window)
    
    sims = 1
    run_time = 0
    for i in range(sims):
        problem.solve()
        run_time += problem.solver_stats.solve_time
#     run_times.append(run_time/sims)
#     objectives.append(objective.value)
    print("Time Window:", time_window)
    print("Run Time:", run_time/sims)
    print("Objective Value:", problem.objective.value)
    
    x_val = spa.dok_matrix((tp.num_nodes, tp.num_nodes))
    for a in range(tp.num_arcs):
        x_val[tp.arcs[a]] = x[a].value
    print(x_val.tocoo())

# plt.figure(1)
# plt.plot(window_mins, run_times);
# plt.title("Run Times");
# plt.xlabel("Time Window");
# plt.ylabel("Run Time");
# plt.figure(2)
# plt.plot(window_mins, objectives);
# plt.title("Profit");
# plt.xlabel("Time Window");
# plt.ylabel("Profit");

NameError: name 'numpy' is not defined

### Without Time Window Constraints

In [ ]:
tp = TP(data)
x, problem = tp.problem_fixed()

sims = 1
run_time = 0
for i in range(sims):
    problem.solve()
    run_time += problem.solver_stats.solve_time
#     run_times.append(run_time/sims)
#     objectives.append(objective.value)
print("Run Time:", run_time/sims)
print("Objective Value:", problem.objective.value)

x_val = spa.dok_matrix((tp.num_nodes, tp.num_nodes))
for a in range(tp.num_arcs):
    x_val[tp.arcs[a]] = x[a].value
print(x_val.tocoo())

# plt.figure(1)
# plt.plot(window_mins, run_times);
# plt.title("Run Times");
# plt.xlabel("Time Window");
# plt.ylabel("Run Time");
# plt.figure(2)
# plt.plot(window_mins, objectives);
# plt.title("Profit");
# plt.xlabel("Time Window");
# plt.ylabel("Profit");

## Map
Needs updating

In [ ]:
x_val = spa.dok_matrix((n+taxis+2, n+taxis+2))
for i in range(n+taxis+2):
    for j in s_list[i]:
        x_val[i,j] = x[arcs.get((i,j))].value
print("Time Window:", time_window)
print(x_val.tocoo())
print(objective.value)

In [ ]:
arcs = np.asarray(np.nonzero(x_val))
# locs = np.unique(arcs[arcs < 200])
# lons = data.iloc[locs, 3]
# lats = data.iloc[locs, 4]

loc_origin = arcs[0][:-taxis-2]
loc_destin = arcs[1][:-taxis-2]

lon_origin = data.iloc[loc_origin, 3].to_numpy()
lat_origin = data.iloc[loc_origin, 4].to_numpy()
lon_destin = data.iloc[loc_destin, 3].to_numpy()
lat_destin = data.iloc[loc_destin, 4].to_numpy()

In [ ]:
# from shapely.geometry import Point, LineString
from matplotlib.patches import FancyArrowPatch

nyc = gpd.read_file(gpd.datasets.get_path('nybb'))
# nyc = nyc.to_crs(epsg=3857)
nyc = nyc.to_crs(epsg=4326)
ax = nyc.plot(figsize=(20, 20), alpha=0.5, edgecolor='k')
ctx.add_basemap(ax)

ax.set_xlim(-74.05, -73.85)
ax.set_ylim(40.68, 40.82)

# nodes = gpd.GeoDataFrame(geometry=gpd.points_from_xy(lons, lats))
# nodes.plot(color='green')

fig = plt.figure()

# One Taxi's Journey
taxi_num = n + 1
node = arcs[1][np.where(arcs[0] == taxi_num)] # First node (rider) for given taxi
while True:
    where = np.array(np.where(loc_origin == node))
    if (where.size == 0): break
    i = where[0][0]
    dx = lon_destin[i] - lon_origin[i]
    dy = lat_destin[i] - lat_origin[i]
    ax.arrow(lon_origin[i], lat_origin[i], dx, dy, color = 'pink', width=.0004)
    node = loc_destin[i]

plt.show()

In [ ]:
class TP():
    def __init__(self, data, taxis=3, taxi_lon=-73.9772, taxi_lat=40.7527, mph=20, driving_cost=.6, taxi_cost=0, flex=timedelta(minutes=20)):
        
        n = len(data.index)
        source = n+taxis
        sink = n+taxis+1
        num_nodes = n+taxis+2
        
        possible = np.zeros((num_nodes, num_nodes))  # =1 if connection (i,j) is possible, =0 if impossible
        profits = np.zeros((num_nodes, num_nodes))   # Net profit from accepting passenger j (based on preceding passenger i)
                                                     # Note: =0 if connection (i,j) is impossible
    
        for i in range(n):
            possible[i, sink] = 1
#             dist_between = haversine(data.loc[i, 'dropoff_longitude'], data.loc[i, 'dropoff_latitude'], taxi_lon, taxi_lat)
#             profits[i, sink] = -driving_cost * dist_between
            for j in range(n):
                if (i == j): continue
                dist_between = haversine(data.loc[i,'dropoff_longitude'], data.loc[i, 'dropoff_latitude'],
                                              data.loc[j, 'pickup_longitude'], data.loc[j, 'pickup_latitude'])
                time_between = timedelta(hours = dist_between / mph)
                if (data.loc[i, 'dropoff_datetime'] + time_between > data.loc[j, 'pickup_datetime'] + flex): continue
                else: possible[i,j] = 1
                profits[i,j] = data.loc[j, 'total_amount'] - driving_cost * (dist_between + data.loc[j, 'trip_distance'])

        for k in range(n, n+taxis):
            possible[source, k] = 1
            profits[source, k] = -taxi_cost
            for j in range(n):
                dist_between = haversine(taxi_lon, taxi_lat, data.loc[j, 'pickup_longitude'], data.loc[j, 'pickup_latitude'])
                time_between = timedelta(hours = dist_between / mph)
                if (midnight + time_between > data.loc[j, 'pickup_datetime'] + flex): continue
                else: possible[k,j] = 1
                profits[k,j] = data.loc[j, 'total_amount'] - driving_cost * (dist_between + data.loc[j, 'trip_distance'])
        
        arcs = {}
        a = 0
        for i in range(num_nodes):
            for j in range(i, num_nodes):
                if (possible[i,j] == 1):
                    arcs[a] = (i,j)
                    a += 1
                if (possible[j,i] == 1):
                    arcs[a] = (j,i)
                    a += 1
        
        self.data = data
        self.n = n
        self.num_taxis = taxis
        self.source = source
        self.sink = sink
        self.num_nodes = num_nodes
        self.taxi_lon = taxi_lon
        self.taxi_lat = taxi_lat
        self.mph = mph
        self.profits = profits
        self.arcs = arcs
        self.num_arcs = len(arcs)
        
    def flow_vars(self):
        A_in = spa.dok_matrix((self.num_nodes, self.num_arcs))
        A_out = spa.dok_matrix((self.num_nodes, self.num_arcs))
        p = np.zeros(self.num_arcs)
        for a in range(self.num_arcs):
            i = self.arcs[a][0]
            j = self.arcs[a][1]
            A_in[i, a] = 1
            A_out[j, a] = 1
            p[a] = self.profits[i, j]
        A_net = A_in - A_out
        
        e = np.zeros(self.num_nodes)
        e[self.source] = self.num_taxis
        e[self.sink] = -self.num_taxis
        
        return A_net, A_in, e, p
    
    def problem_fixed(self):
        A, A_in, e, p = self.flow_vars()

        # Maximum Flow Problem CVXPY
        x = cp.Variable(self.num_arcs, boolean = True)

        ## Network Flow Constraints
        constraints = [A @ x == e]
        inflow = A_in @ x
        constraints += [inflow[:self.source] <= 1]
        
        profit = np.transpose(p) @ x
        objective = cp.Maximize(profit)
        problem = cp.Problem(objective, constraints)
        return x, problem
    
    def time_vars(self, time_window):
        B = spa.dok_matrix((self.num_arcs, self.num_arcs))
        d = np.zeros(self.num_arcs)
        t_min, t_max = self.time_cons(time_window)
        for a in range(self.num_arcs):
            i = self.arcs[a][0]
            j = self.arcs[a][1]
            B[a, a] = t_max[i] - t_min[j] + self.T(i, j)
            d[a] = t_max[i] - t_min[j]
        return B, d
    
    def time_cons(self, time_window):
        t_min = []
        t_max = []
        for i in range(self.n):
            t_min.append(to_minutes(self.data.loc[i, 'pickup_datetime'] - midnight))
            t_max.append(to_minutes(self.data.loc[i, 'pickup_datetime'] - midnight + time_window))
        for k in range(self.n, self.num_nodes):
            t_min.append(0)
            t_max.append(0)
        return t_min, t_max
    
    def T(self, i, j):
        if (i == j or i == self.source): return 0
        # From customer to sink
        if (j == self.sink): return -9999999
        # From taxi to customer
        if (i >= self.n):
            dist_between = haversine(self.taxi_lon, self.taxi_lat, self.data.loc[j, 'pickup_longitude'], self.data.loc[j, 'pickup_latitude'])
            output = timedelta(hours = dist_between / self.mph)
        # From customer to customer
        else:
            dist_between = haversine(self.data.loc[i,'dropoff_longitude'], self.data.loc[i, 'dropoff_latitude'],
                                      self.data.loc[j, 'pickup_longitude'], self.data.loc[j, 'pickup_latitude'])
            time_between = timedelta(hours = dist_between / self.mph)
            output = data.loc[i, 'dropoff_datetime'] - data.loc[i, 'pickup_datetime'] + time_between
        return to_minutes(output)
    
    def problem(self, time_window):
        A, A_in, e, p = self.flow_vars()
        B, d = self.time_vars(time_window)
        C = np.transpose(A[:self.n,])

        # Maximum Flow Problem CVXPY
        x = cp.Variable(self.num_arcs, boolean = True)
        t = cp.Variable(self.n)

        ## Network Flow Constraints
        constraints = [A @ x == e]
        inflow = A_in @ x
        constraints += [inflow[:self.source] <= 1]
        ## Time Window Constraints
        t_min, t_max = self.time_cons(time_window)
        constraints += [t_min[:self.n] <= t, t <= t_max[:self.n]]
        constraints += [B @ x + C @ t <= d]

        profit = np.transpose(p) @ x
        objective = cp.Maximize(profit)
        problem = cp.Problem(objective, constraints)
        return x, problem
    
    # Read about @property